In [1]:
import requests
import pandas as pd
import json
from pyhocon import ConfigFactory
import os
import time
import numpy as np

In [2]:
conf = ConfigFactory.parse_file('API_Keys.conf')
key = conf.get_string('API.covalent.key')

In [3]:
#wallet parquet file path
pths = r'D:\Teller_Finance\wallet_info_from_covalent\wallet_raw_df.parquet.gzip'

In [4]:
#read all wallet data
df_wallet=pd.read_parquet(pths)

#get address
wallet_address=df_wallet['address'].unique()

In [2]:
def create_api_string(api, input_dict=None):
    try:
        return api.format(**input_dict)
    except:
        print('Failed to Create Api String')

def secs():
    return np.random.randint(1,5)

In [10]:
class api_data_getter:
    
    def __init__(self,**kwargs):
        self.kwargs = kwargs
        
    
    def covalent_api(self):
        '''
                Parameters:
                        wallet_address_list (list): list of wallets
                        blockchain_data (str): e.g. balances_v2
                        params (dict) : params={"limit":"100", "key":key } key is necessary
                        number_of_wallets (int) : the number of wallets to loop through default is the full wallet_address_list
                        api_string (str) : api string e.g. "https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"

                Returns:
                        api dataframe output for the requested blockchain_data
        '''
        
        wallet_address = self.kwargs.get('wallet_address_list')
        blockchain_data = self.kwargs.get('blockchain_data') #balances_v2
        params = self.kwargs.get('params') 
        if self.kwargs.get('number_of_wallets'):
            n = self.kwargs.get('number_of_wallets')
        else:
            n= len(wallet_address)# use all wallets
        
        if self.kwargs.get('api_string'):
            api_string = self.kwargs.get('api_string') #params={"limit":"100", "key":key }
        else:
            #some defualt for now
            api_string = "https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"
            
        
        #loop through individual wallets and get data
        df_res= pd.DataFrame()
        for addr in wallet_address[0:n]:
            time.sleep(secs())
            input_dict = {"address":addr, "blockchain_data" :blockchain_data}
            api_= create_api_string(api_string,input_dict)
            r = requests.get(api_ ,params = params)
            #print('status of the api request is...')
            #print(r)
            #print('\n')
            df = pd.read_json(r.content,orient='index').reset_index().rename(columns={'index':'server_output'})
            df = df.loc[ df['server_output'] == 'data'] # get the data component as opposed to error
            df_res = df_res.append(df)
        self.df_res = df_res
        return self
    
    
    def thegraphql_api(self):
        '''
                Parameters:

                Returns:
                        api dataframe output for the requested blockchain_data
        '''
        
        query = self.kwargs.get('query')
        if self.kwargs.get('api_string'):
            api_string = self.kwargs.get('api_string')
        else:
            api_string = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"
            
        r = requests.post(api_string, json={'query': query})
        df = pd.read_json(r.content,orient='index').reset_index().rename(columns={'index':'server_output'})
        df = df.loc[ df['server_output'] == 'data'] # get the data component as opposed to error
        self.df_res = df
        return self
        
    
    def save_to_parquet(self,file_name):
        self.df_res = self.df_res.reset_index(drop=True)
        self.df_res.to_parquet(file_name + '.gzip' ,compression='gzip')
        
    def save_to_json(self,file_name):
        self.df_res = self.df_res.reset_index(drop=True)
        self.df_res.to_json(file_name + '.json' ,compression='gzip')
    

In [11]:
#example run graphql
query  = """{
  pairs {
    id
  }
}"""

input_dict= {'query':query,'api_string':"https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"}

adg = api_data_getter(**input_dict)
adg.thegraphql_api()
try:
    adg.save_to_parquet('graphql')
except:
    adg.save_to_json('graphql')

In [7]:
# #example run
# params = {"limit":"1000", "key":key }
# #input_dict= {'wallet_address_list':wallet_address,'blockchain_data':'balances_v2','params':params,'number_of_wallets':None,'api_string':"https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"}
# input_dict= {'wallet_address_list':wallet_address,'blockchain_data':'transactions_v2','params':params,'number_of_wallets':1000,'api_string':"https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"}
# adg = api_data_getter(**input_dict)
# adg.covalent_api()
# try:
#     adg.save_to_parquet('transactions')
# except:
#     adg.save_to_json('transaction')